In [43]:
import numpy as np
import pandas as pd
import reverse_geocoder as rg 
import pprint 
from datetime import datetime,time, date
import requests
import networkx as nx
import json
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
import time

In [2]:
trip_df = pd.read_csv('2016_june_cleaned.csv')

In [3]:
trip_df.shape

(51607, 14)

In [4]:
trip_df = trip_df.rename(columns = {'Unnamed: 0': 'trip_id'})

In [5]:
trip_df.head()

,trip_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,total_amount,dropoff_lat_long_tuple,dropoff_city
0,0,2016-06-01 00:00:55,2016-06-01 00:31:25,1,19.20,-73.790199,40.646770,2,-73.966484,40.789272,1,66.34,"(40.78927230834961, -73.96648406982422)",Manhattan
1,1,2016-06-01 00:01:35,2016-06-01 00:34:08,1,18.52,-73.789909,40.646751,2,-73.962631,40.801632,1,58.34,"(40.80163192749024, -73.96263122558594)",Manhattan
2,2,2016-06-01 00:01:56,2016-06-01 00:34:33,1,21.37,-73.782372,40.644585,2,-73.979279,40.776905,1,70.01,"(40.77690505981445, -73.97927856445312)",Manhattan
3,3,2016-06-01 00:02:36,2016-06-01 00:33:02,1,16.91,-73.789429,40.646759,2,-73.979111,40.766800,1,63.36,"(40.76679992675781, -73.97911071777342)",Manhattan
4,4,2016-06-01 00:03:54,2016-06-01 00:31:41,1,17.40,-73.789833,40.646851,2,-73.971542,40.762051,1,72.96,"(40.7620506286621, -73.97154235839844)",Manhattan


In [ ]:
trip_df.

In [6]:
trip_df['language'] = 1

In [7]:
trip_df['profession'] = 1

In [8]:
v = trip_df['language'] == 1
trip_df.loc[v,'language'] = np.random.choice((0,1), v.sum(), p=(0.5,0.5))

In [9]:
v = trip_df['profession'] == 1
trip_df.loc[v,'profession'] = np.random.choice((0,1), v.sum(), p=(0.5,0.5))

In [10]:
trip_df.tail()

,trip_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,total_amount,dropoff_lat_long_tuple,dropoff_city,language,profession
51602,51602,2016-06-30 23:56:12,2016-07-01 00:33:46,2,17.60,-73.782288,40.644569,2,-73.980667,40.764435,2,52.80,"(40.76443481445313, -73.98066711425781)",Manhattan,0,0
51603,51603,2016-06-30 23:59:04,2016-07-01 00:58:31,2,18.10,-73.781990,40.644711,2,-73.986450,40.758171,1,69.99,"(40.75817108154297, -73.9864501953125)",Manhattan,0,1
51604,51604,2016-06-30 23:59:20,2016-07-01 00:45:34,1,21.22,-73.790337,40.646629,2,-73.984230,40.779942,2,58.34,"(40.77994155883789, -73.9842300415039)",Manhattan,1,0
51605,51605,2016-06-30 23:59:53,2016-07-01 00:32:22,1,16.70,-73.789238,40.646832,2,-73.972740,40.764446,2,52.80,"(40.76444625854492, -73.97274017333984)",Manhattan,0,1
51606,51606,2016-06-30 23:59:54,2016-07-01 00:39:51,2,17.90,-73.784592,40.648472,2,-73.974915,40.758846,2,58.34,"(40.75884628295898, -73.97491455078125)",Manhattan,0,1


In [34]:
all_trips_sum = sum(trip_df.trip_distance)

In [11]:
trip_df.tpep_pickup_datetime = trip_df.tpep_pickup_datetime.apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
trip_df.tpep_dropoff_datetime = trip_df.tpep_dropoff_datetime.apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
trip_df = trip_df.sort_values(by = 'tpep_pickup_datetime')
trip_df.head()

,trip_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,total_amount,dropoff_lat_long_tuple,dropoff_city,language,profession
0,0,2016-06-01 00:00:55,2016-06-01 00:31:25,1,19.20,-73.790199,40.646770,2,-73.966484,40.789272,1,66.34,"(40.78927230834961, -73.96648406982422)",Manhattan,0,0
1,1,2016-06-01 00:01:35,2016-06-01 00:34:08,1,18.52,-73.789909,40.646751,2,-73.962631,40.801632,1,58.34,"(40.80163192749024, -73.96263122558594)",Manhattan,1,1
2,2,2016-06-01 00:01:56,2016-06-01 00:34:33,1,21.37,-73.782372,40.644585,2,-73.979279,40.776905,1,70.01,"(40.77690505981445, -73.97927856445312)",Manhattan,0,1
3,3,2016-06-01 00:02:36,2016-06-01 00:33:02,1,16.91,-73.789429,40.646759,2,-73.979111,40.766800,1,63.36,"(40.76679992675781, -73.97911071777342)",Manhattan,1,1
4,4,2016-06-01 00:03:54,2016-06-01 00:31:41,1,17.40,-73.789833,40.646851,2,-73.971542,40.762051,1,72.96,"(40.7620506286621, -73.97154235839844)",Manhattan,1,0


In [12]:
trip_df.head()

,trip_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,total_amount,dropoff_lat_long_tuple,dropoff_city,language,profession
0,0,2016-06-01 00:00:55,2016-06-01 00:31:25,1,19.20,-73.790199,40.646770,2,-73.966484,40.789272,1,66.34,"(40.78927230834961, -73.96648406982422)",Manhattan,0,0
1,1,2016-06-01 00:01:35,2016-06-01 00:34:08,1,18.52,-73.789909,40.646751,2,-73.962631,40.801632,1,58.34,"(40.80163192749024, -73.96263122558594)",Manhattan,1,1
2,2,2016-06-01 00:01:56,2016-06-01 00:34:33,1,21.37,-73.782372,40.644585,2,-73.979279,40.776905,1,70.01,"(40.77690505981445, -73.97927856445312)",Manhattan,0,1
3,3,2016-06-01 00:02:36,2016-06-01 00:33:02,1,16.91,-73.789429,40.646759,2,-73.979111,40.766800,1,63.36,"(40.76679992675781, -73.97911071777342)",Manhattan,1,1
4,4,2016-06-01 00:03:54,2016-06-01 00:31:41,1,17.40,-73.789833,40.646851,2,-73.971542,40.762051,1,72.96,"(40.7620506286621, -73.97154235839844)",Manhattan,1,0


In [13]:
trip_df.tail()

,trip_id,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,total_amount,dropoff_lat_long_tuple,dropoff_city,language,profession
51602,51602,2016-06-30 23:56:12,2016-07-01 00:33:46,2,17.60,-73.782288,40.644569,2,-73.980667,40.764435,2,52.80,"(40.76443481445313, -73.98066711425781)",Manhattan,0,0
51603,51603,2016-06-30 23:59:04,2016-07-01 00:58:31,2,18.10,-73.781990,40.644711,2,-73.986450,40.758171,1,69.99,"(40.75817108154297, -73.9864501953125)",Manhattan,0,1
51604,51604,2016-06-30 23:59:20,2016-07-01 00:45:34,1,21.22,-73.790337,40.646629,2,-73.984230,40.779942,2,58.34,"(40.77994155883789, -73.9842300415039)",Manhattan,1,0
51605,51605,2016-06-30 23:59:53,2016-07-01 00:32:22,1,16.70,-73.789238,40.646832,2,-73.972740,40.764446,2,52.80,"(40.76444625854492, -73.97274017333984)",Manhattan,0,1
51606,51606,2016-06-30 23:59:54,2016-07-01 00:39:51,2,17.90,-73.784592,40.648472,2,-73.974915,40.758846,2,58.34,"(40.75884628295898, -73.97491455078125)",Manhattan,0,1


In [14]:
# trip_df['pickup_date'] = trip_df.tpep_pickup_datetime.apply(lambda date_time: date_time.date())
# trip_df['pickup_time'] = trip_df.tpep_pickup_datetime.apply(lambda date_time: date_time.time())

In [15]:
trip_df = trip_df[(trip_df.tpep_pickup_datetime >= datetime(2016, 6, 10, 8,0,0)) 
                  & (trip_df.tpep_pickup_datetime <= datetime(2016, 6, 10, 22,0,0))]

In [40]:
total_trips = trip_df.shape[0]

In [17]:
B_hdist = []
averageSpeed = None
B_tuple = None
allRouteDistance = dict()
totalRouteDistance = 0

In [18]:
# Pooling Window = 5 mins

def getGroup(a,b):
    res = trip_df[((trip_df.tpep_pickup_datetime - b).apply(lambda x: (x.total_seconds() <= 300) & 
                                                          (x.total_seconds() > 0))) & (trip_df.trip_id != a)]
    return res.trip_id.values


In [19]:
trip_df["time"]=trip_df.tpep_dropoff_datetime-trip_df.tpep_pickup_datetime;
trip_df["time"] = trip_df["time"].apply(lambda x: x.days*24*60*60+x.seconds)  
trip_df = trip_df[trip_df["time"] != 0]
trip_df["speed"]=trip_df.trip_distance/trip_df["time"]
averageSpeed=trip_df["speed"].mean()
#print(averageSpeed)

In [20]:
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 3956  # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [21]:
def timeAtoB(hdist):
    return hdist/averageSpeed;


In [22]:
def lowerBound(B_hdist, B_tuple):
    res=[]
    for i in range(len(B_tuple)):
        b=B_tuple[i]
        timeBDest=trip_df.tpep_dropoff_datetime[b-1]-trip_df.tpep_pickup_datetime[b-1];  
        timeBDest=timeBDest.days*24*60*60+timeBDest.seconds; 
        delay=.20*trip_df.trip_distance[b-1]
        lval=timeADest+timeAtoB(B_hdist[i])
        rval=timeBDest+delay

        if(lval < rval):
            res.append((a,b))
    return res

In [23]:
def startComputation(B_tuple, trip_id):
#     print("Calculating all haversine distances for B's") 
    hDistA=(haversine(trip_df.pickup_longitude[row.trip_id],trip_df.pickup_latitude[row.trip_id],trip_df.dropoff_longitude[row.trip_id],trip_df.dropoff_latitude[row.trip_id]))
    for b in B_tuple:
        B_hdist.append(abs(haversine(trip_df.pickup_longitude[b-1],trip_df.pickup_latitude[b-1],trip_df.dropoff_longitude[b-1],trip_df.dropoff_latitude[b-1])-hDistA)) 
#     print(B_hdist)
    
#     print("Applying lower bound. Find the answer below:")
    result = lowerBound(B_hdist, B_tuple)
#     print(result)
    return result

In [24]:
def getActualDistance(tuple_list):
    
    result = []    #(drop-off_A_trip_id, drop-off_B_trip_id, actual distance from drop_off_A to drop-off_B)
    
    for pair in tuple_list:
        url = "https://graphhopper.com/api/1/matrix?"
        points = "point="+str(trip_df.loc[pair[0], "dropoff_latitude"])+","+str(trip_df.loc[pair[0], "dropoff_longitude"])
        points += "&point="+str(trip_df.loc[pair[1], "dropoff_latitude"])+","+str(trip_df.loc[pair[1], "dropoff_longitude"])
        url += points
        url+="&type=json&vehicle=car&debug=true&out_array=weights&out_array=times&out_array=distances&key=c845511d-5ce0-4afd-82e3-daac6de103e5"
        response = requests.get(url)
        response_dict = json.loads(response.text)
#         print(response.text)
#         print(response_dict['distances'])
        actual_dist = response_dict['distances'][0][1] * 0.000621371    # 1 meter = 0.000621371 miles
#         print(actual_dist)    
        result.append((pair[0], pair[1], actual_dist, trip_df.trip_distance[pair[0]], trip_df.trip_distance[pair[1]], trip_df.language[pair[0]], trip_df.language[pair[1]], trip_df.profession[pair[0]], trip_df.profession[pair[1]]))
    
    return result 
    

In [25]:
def getSocialScore(prof1, prof2, lang1, lang2):
    if lang1== lang2:
        if prof1 == prof2:
            return 1
        else:
            return 0.6
    else:
        return 0.1

In [26]:
def getDistanceSaved(distance1, distance2, routeDistance):
    dist = (distance1+distance2) - routeDistance
    return dist

In [27]:
def getWeight(distSaved, socialScore):
    weight = (0.8*distSaved) + (0.2*socialScore)
    return round(weight,3)

In [31]:
trips_covered = set()

total_sum = 0

trips_merged = []



start_time = time.time()


for i,row in trip_df.iterrows():
#     print("********___" + str(row.trip_id) + "___**********")
    timeADest = row.tpep_dropoff_datetime - row.tpep_pickup_datetime
    timeADest = timeADest.days*24*60*60+timeADest.seconds
    a=row.trip_id
#     print("Getting all Bs")
    B_tuple= getGroup(row.trip_id, row.tpep_pickup_datetime)
#     print(B_tuple)
    B_tuple = [val for val in B_tuple if val not in trips_covered]
    tuple_list = startComputation(B_tuple, row.trip_id)
    actual_dist_values = getActualDistance(tuple_list)
    actual_dist_values     # in miles
    if(len(actual_dist_values) != 0):
        T = np.append(a, B_tuple).tolist()
        linkWeights = []
        for row in actual_dist_values:
            distanceA = row[3]
            distanceB = row[4]
            routeDistance = row[2] + row[3]
            prof1 = row[7]
            prof2 = row[8]
            lang1 = row[5]
            lang2 = row[6]
            allRouteDistance[row[0],row[1]] = routeDistance
            
            socialScore = getSocialScore(prof1, prof2, lang1, lang2)
            distanceSaved = getDistanceSaved(distanceA, distanceB, routeDistance)
            if distanceSaved >= 0:
                finalWeight = getWeight(distanceSaved, socialScore)
                linkWeights.append((row[0], row[1], finalWeight))
#         print(linkWeights)
        if (len(linkWeights) != 0):
            G = nx.Graph()
            G.add_nodes_from(T)
            for link in linkWeights:
                G.add_edge(link[0], link[1], weight=link[2])

#             nx.draw(G, with_labels=True)
            M_max = nx.max_weight_matching(G, weight='weight')
#             print(M_max)
            list_M_max = list(M_max)
    

#             print(allRouteDistance)
            for l in list_M_max:
#                 print(l)
#                 print(type)
                if allRouteDistance.get(l):
                    totalRouteDistance += round(float(allRouteDistance.get(l)), 2)
                else:
                    totalRouteDistance += round(float(allRouteDistance.get(l[::-1])), 2)
#                     totalRouteDistance += float(allRouteDistance.get(l[::-1]))
        
#             print(list_M_max)
            trips_merged.append(list_M_max)
#             
            trips_covered.add(list_M_max[0][0])
            trips_covered.add(list_M_max[0][1])
            total_sum += trip_df.trip_distance[list_M_max[0][0]]
            total_sum += trip_df.trip_distance[list_M_max[0][1]]
        
        
        
            print(trips_merged)
            print(totalRouteDistance)
end_time = time.time()
# print(trips_merged)
    


    
    

[[(16311, 16303)]]
140.81
[[(16311, 16303)], [(16304, 16310)]]
158.35
[[(16311, 16303)], [(16304, 16310)], [(16305, 16306)]]
177.97


KeyboardInterrupt: 

In [33]:
distanceSavedPercent = ((total_sum - totalRouteDistance) * 100)/ all_trips_sum

NameError: name 'total_sum' is not defined

In [42]:
total_merged_trips = len(trips_covered)
tripsSavedPercent = (total_merged_trips*100)/(2*total_trips)

0

In [37]:
total_computation_time = end - start

3

In [ ]:
# dist_plot = [distanceSavedPercent,  ]
# trip_plot = [tripsSavedPercent, ]

